In [16]:
import pandas as pd
import re

In [17]:

def precise_clean_jd_v3(row):
    # 统一处理文本，确保是字符串
    text = str(row['内容全文']) if pd.notna(row['内容全文']) else ""
    
    # --- 1. 数据截断：删除指定内容之后的部分 ---
    # 找到截断标志的起始位置
    cutoff_marker = "応募する あとからチェックしたい方は 気になる"
    if cutoff_marker in text:
        text = text.split(cutoff_marker)[0]
    
    # --- 2. 职位与公司提取（基于空格划分） ---
    # 步骤：删掉开头的 "NEW "
    header_area = re.sub(r'^NEW\s+', '', text.strip())
    
    # 按照空格拆分。split() 默认会处理连续空格并返回非空片段
    parts = header_area.split()
    
    # 根据你的要求：第一个空格前是职位，第二个前是公司
    job_title = parts[0] if len(parts) > 0 else "未知职位"
    company_name = parts[1] if len(parts) > 1 else "未知公司"

    # --- 3. 精准板块提取函数 ---
    def extract_between(full_text, start_kw, end_kw):
        if start_kw not in full_text:
            return ""
        # 找到起点位置
        start_idx = full_text.find(start_kw) + len(start_kw)
        # 从起点开始往后找终点
        if end_kw:
            end_idx = full_text.find(end_kw, start_idx)
            if end_idx != -1:
                return full_text[start_idx:end_idx].strip()
        return full_text[start_idx:].strip()

    # 按你指定的范围进行切片
    job_description = extract_between(text, "仕事の内容", "配属先情報")
    department_info = extract_between(text, "配属先情報", "必要な能力・経験")
    requirements    = extract_between(text, "必要な能力・経験", "勤務地")
    selection_info  = extract_between(text, "選考内容", "企業情報")

    # --- 4. 辅助信息提取 ---
    # 员工人数
    emp_match = re.search(r'従業員数\s*([\d,]+名)', text)
    employee_count = emp_match.group(1) if emp_match else "未知"
    
    # 在宅信息（取 制度・福利厚生 附近的描述）
    remote_match = re.search(r'[^。]{0,20}(在宅|リモート|テレワーク)[^。]{0,20}', text)
    remote_info = remote_match.group(0).strip() if remote_match else "未提到"

    return pd.Series([
        job_title, 
        company_name, 
        employee_count, 
        remote_info, 
        job_description, 
        department_info, 
        requirements, 
        selection_info
    ])

# --- 主程序 ---
df = pd.read_csv('jd_data_raw.csv')

new_cols = [
    '职位名称', '公司名', '员工人数', '在宅信息', 
    '工作内容', '配属先情报', '能力经验要求', '选考流程'
]

# 执行清洗
df[new_cols] = df.apply(precise_clean_jd_v3, axis=1)

# 整理列顺序
final_cols = ['序号', '求人票ID', '更新时间'] + new_cols
final_df = df[final_cols]


In [18]:
display(final_df.head(2))

,序号,求人票ID,更新时间,职位名称,公司名,员工人数,在宅信息,工作内容,配属先情报,能力经验要求,选考流程
0,1394,j107579424,2026年01月09日,【データ活用/教育枠】佐川急便が抱えるビジネス課題をデータ分析の力で解決,ＳＧシステム株式会社,"2,634名",【魅力】リモートワーク基本 ★入社者のスムーズな定着を目,佐川急便が抱えるビジネス課題を、データ分析の力で解決に導くプロジェクトをお任せします！SQL...,グループDX推進1部 IT業務改革課 ※全体16名 ※現在30テーマ程が走っており、1テーマ...,【必須】・SQLを用いたデータ抽出・加工の実務経験 ・Pythonを用いたデータ分析・機械学...,
1,1395,t107575547,2026年01月09日,【4/1入社限定】データアナリスト/基盤構築エンジニア/自由な働き方可能◎,ＡＭＢＬ株式会社,700名,煙（屋外喫煙可能場所あり） 勤務地備考 在宅勤務可※在宅勤務/リモートワークの利用に,多様な業界で、データ分析基盤の構築から顧客分析・需要予測・価格最適化・市場/売上分析・KPI...,イノベーション事業本部 データサイエンス事業部,【必須】■IT業界での就業経験 （目安：2年以上） 【いずれか必須(1年以上)】■システム開...,


In [19]:
# 保存
final_df.to_csv('jd_data_structured.csv', index=False, encoding='utf-8-sig')